## Integrating Datasets and Engineering Features

This notebook contains examples and code from various sources:

<a href="github.com/alicezheng/feature-engineering-book">Feature Engineering for Machine Learning Book Code Repository </a>
<br><a href="github.com/jakevdp/PythonDataScienceHandbook"> Python Data Science Handbook Code Repository </a>

The goal of this notebook is to illustrate how integration of multiple datasets can be done and how features can be engineered.

Detailed explanations for important code snippets are provided by Mervat Abuelkheir as part of the CSEN1095 Data Engineering Course.

Pay attention to the <span style="color:red"> <b> paragraphs in bold red</b></span>; they ask you to do something and provide input!


For convenience, we will start by redefining the `display()` functionality from the previous section:


In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
%matplotlib inline

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## Part 1: Merging and Joining Datasets

One essential feature offered by Pandas is its high-performance, in-memory join and merge operations.
If you have ever worked with databases, you should be familiar with this type of data interaction.
The main interface for this is the ``pd.merge`` function, and we'll see few examples of how this can work in practice.

### Relational Algebra

The behavior implemented in ``pd.merge()`` is a subset of what is known as *relational algebra*, which is a formal set of rules for manipulating relational data, and forms the conceptual foundation of operations available in most databases.
The strength of the relational algebra approach is that it proposes several primitive operations, which become the building blocks of more complicated operations on any dataset.
With this lexicon of fundamental operations implemented efficiently in a database or other program, a wide range of fairly complicated composite operations can be performed.

Pandas implements several of these fundamental building-blocks in the ``pd.merge()`` function and the related ``join()`` method of ``Series`` and ``Dataframe``s.
As we will see, these let you efficiently link data from different sources.

### Categories of Joins

The ``pd.merge()`` function implements a number of types of joins: the *one-to-one*, *many-to-one*, and *many-to-many* joins.
All three types of joins are accessed via an identical call to the ``pd.merge()`` interface; the type of join performed depends on the form of the input data.
Here we will show simple examples of the three types of merges, and discuss detailed options further below.

#### One-to-one joins

Perhaps the simplest type of merge expresion is the one-to-one join, which is in many ways very similar to the column-wise concatenation.

As a concrete example, consider the following two ``DataFrames`` which contain information on several employees in a company:

In [2]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


To combine this information into a single ``DataFrame``, we can use the ``pd.merge()`` function. 

The ``pd.merge()`` function recognizes that each ``DataFrame`` has an "employee" column, and automatically joins using this column as a key.
The result of the merge is a new ``DataFrame`` that combines the information from the two inputs.
Notice that the order of entries in each column is not necessarily maintained: in this case, the order of the "employee" column differs between ``df1`` and ``df2``, and the ``pd.merge()`` function correctly accounts for this.

Additionally, keep in mind that the merge in general discards the index, except in the special case of merges by index (see the ``left_index`` and ``right_index`` keywords, discussed momentarily).

In [3]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


#### Many-to-one joins

Many-to-one joins are joins in which one of the two key columns contains duplicate entries.
For the many-to-one case, the resulting ``DataFrame`` will preserve those duplicate entries as appropriate. 
Consider the following example of a many-to-one join:

In [4]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

The resulting ``DataFrame`` will have an aditional column with the "supervisor" information, where the information is repeated in one or more locations as required by the inputs.

#### Many-to-many joins

Many-to-many joins are a bit confusing conceptually, but are nevertheless well defined.
If the key column in both the left and right array contains duplicates, then the result is a many-to-many merge.
This will be perhaps most clear with a concrete example.
Consider the following, where we have a ``DataFrame`` showing one or more skills associated with a particular group.
By performing a many-to-many join, we can recover the skills associated with any individual person:

In [5]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


 In practice, datasets are rarely as clean as the one we worked with above. In the following section we'll consider some of the options provided by `pd.merge()` that enable you to tune how the join operations work.

### Specification of the Merge Key

We've already seen the default behavior of ``pd.merge()``: it looks for one or more matching column names between the two inputs, and uses this as the key.
However, often the column names will not match so nicely, and ``pd.merge()`` provides a variety of options for handling this.

#### The ``on`` keyword

Most simply, you can explicitly specify the name of the key column using the ``on`` keyword, which takes a column name or a list of column names. This option works only if both the left and right ``DataFrame``s have the specified column name.

In [6]:
display('df1', 'df2', "pd.merge(df1, df2, on='employee')")

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2, on='employee')
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

#### The ``left_on`` and ``right_on`` keywords

At times you may wish to merge two datasets with different column names; for example, we may have a dataset in which the employee name is labeled as "name" rather than "employee".
In this case, we can use the ``left_on`` and ``right_on`` keywords to specify the two column names:

In [7]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")')

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1, df3, left_on="employee", right_on="name")
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000

The result has a redundant column that we can drop if desired–for example, by using the ``drop()`` method of ``DataFrame``s:

In [8]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


#### The ``left_index`` and ``right_index`` keywords

Sometimes, rather than merging on a column, you would instead like to merge on an index.
For example, your data might look like this:

In [9]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
display('df1a', 'df2a')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


You can use the index as the key for merging by specifying the ``left_index`` and/or ``right_index`` flags in ``pd.merge()``:

In [10]:
display('df1a', 'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)")

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


For convenience, ``DataFrame``s implement the ``join()`` method, which performs a merge that defaults to joining on indices:

In [11]:
display('df1a', 'df2a', 'df1a.join(df2a)')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


### Specifying Set Arithmetic for Joins

In all the preceding examples we have glossed over one important consideration in performing a join: the type of set arithmetic used in the join.
This comes up when a value appears in one key column but not the other. Consider this example:

In [12]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
display('df6', 'df7', 'pd.merge(df6, df7)')

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7)
   name   food drink
0  Mary  bread  wine

Here we have merged two datasets that have only a single "name" entry in common: Mary.
By default, the result contains the *intersection* of the two sets of inputs; this is what is known as an *inner join*.
We can specify this explicitly using the ``how`` keyword, which defaults to ``"inner"``:

In [13]:
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


Other options for the ``how`` keyword are ``'outer'``, ``'left'``, and ``'right'``.
An *outer join* returns a join over the union of the input columns, and fills in all missing values with NAs:

In [14]:
display('df6', 'df7', "pd.merge(df6, df7, how='outer')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='outer')
     name   food drink
0   Peter   fish   NaN
1    Paul  beans   NaN
2    Mary  bread  wine
3  Joseph    NaN  beer

The *left join* and *right join* return joins over the left entries and right entries, respectively.
For example:

In [15]:
display('df6', 'df7', "pd.merge(df6, df7, how='left')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='left')
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine

The output rows now correspond to the entries in the left input. Using
``how='right'`` works in a similar manner.

### Overlapping Column Names: The ``suffixes`` Keyword

Finally, you may end up in a case where your two input ``DataFrame``s have conflicting column names.
Consider this example:

In [16]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
display('df8', 'df9', 'pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])
   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

## Example: The US States Data

Merge and join operations come up most often when combining data from different sources.
Here we will consider an example of some data about US states and their populations.
The data files can be found in the data folder.

Let's take a look at the three datasets, using the Pandas ``read_csv()`` function:

In [17]:
pop = pd.read_csv('data/state-population.csv')
areas = pd.read_csv('data/state-areas.csv')
abbrevs = pd.read_csv('data/state-abbrevs.csv')

display('pop.head()', 'areas.head()', 'abbrevs.head()')

pop.head()
  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

areas.head()
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

abbrevs.head()
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA

Given this information, say we want to compute a relatively straightforward result: rank US states and territories by their 2010 population density.
We clearly have the data here to find this result, but we'll have to combine the datasets to find the result.

We'll start with a many-to-one merge that will give us the full state name within the population ``DataFrame``.
We want to merge based on the ``state/region``  column of ``pop``, and the ``abbreviation`` column of ``abbrevs``.
We'll use ``how='outer'`` to make sure no data is thrown away due to mismatched labels.

In [18]:
merged = pd.merge(pop, abbrevs, how='outer',
                  left_on='state/region', right_on='abbreviation')
merged = merged.drop('abbreviation', 1) # drop duplicate info
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


Let's double-check whether there were any mismatches here, which we can do by looking for rows with nulls:

In [19]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

Some of the ``population`` info is null; let's figure out which these are!

In [20]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


It appears that all the null population values are from Puerto Rico prior to the year 2000; this is likely due to this data not being available from the original source.

More importantly, we see also that some of the new ``state`` entries are also null, which means that there was no corresponding entry in the ``abbrevs`` key!
Let's figure out which regions lack this match:

In [21]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

We can quickly infer the issue: our population data includes entries for Puerto Rico (PR) and the United States as a whole (USA), while these entries do not appear in the state abbreviation key.
We can fix these quickly by filling in appropriate entries:

In [22]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

No more nulls in the ``state`` column: we're all set!

Now we can merge the result with the area data using a similar procedure.
Examining our results, we will want to join on the ``state`` column in both:

In [23]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


Again, let's check for nulls to see if there were any mismatches:

In [24]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

There are nulls in the ``area`` column; we can take a look to see which regions were ignored here:

In [25]:
#final['state'][final['area (sq. mi)'].isnull()].unique()
final.loc[final['area (sq. mi)'].isnull(), 'state'].unique()

array(['United States'], dtype=object)

We see that our ``areas`` ``DataFrame`` does not contain the area of the United States as a whole.
We could insert the appropriate value (using the sum of all state areas, for instance), but in this case we'll just drop the null values because the population density of the entire United States is not relevant to our current discussion:

In [26]:
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


Let's check if there are any duplicate records in the dataset before we proceed to query it.

In [27]:
final.duplicated()

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
2446    False
2447    False
2468    False
2469    False
2470    False
2471    False
2472    False
2473    False
2474    False
2475    False
2476    False
2477    False
2478    False
2479    False
2480    False
2481    False
2482    False
2483    False
2484    False
2485    False
2486    False
2487    False
2488    False
2489    False
2490    False
2491    False
2492    False
2493    False
2494    False
2495    False
Length: 2476, dtype: bool

Thankfully there are no duplicates. Now we have all the data we need. To answer the question of interest, let's first select the portion of the data corresponding with the year 2000, and the total population.
We'll use the ``query()`` function to do this quickly (this requires the ``numexpr`` package to be installed.

In [28]:
!pip install numexpr

In [29]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


Now let's compute the population density and display it in order.
We'll start by re-indexing our data on the state, and then compute the result:

In [30]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']

In [31]:
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [32]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

## <span style="color:red">Assignment Mini Challenge 1</span>

In the data/microbiome subdirectory, there are 9 spreadsheets of microbiome data that was acquired from RNA sequencing procedures, with a 10th file describing the content of each spreadsheet. 

<span style="color:red"><b>Write code that imports each of the data spreadsheets and combines them into a single `DataFrame`, adding identifying information from the metadata spreadsheet as columns in the combined `DataFrame`.</b></span>

In [36]:
metadata = pd.read_excel('/home/adam/Downloads/CSEN1095-Data-Engineering-master/data/microbiome/metadata.xls', sheet_name='Sheet1')

chunks = []
for i in range(9):
    this_file = pd.read_excel('/home/adam/Documents/DataEngineering/DE/data/microbiome/MID{0}.xls'.format(i+1), 'Sheet 1', index_col=0, header=None, names=['Taxon', 'Count'])
    this_file.columns = ['Count']
    this_file.index.name = 'Taxon'
    for m in metadata.columns:
        this_file[m] = metadata.iloc[i][m]
    chunks.append(this_file)

pd.concat(chunks)

,Count,BARCODE,GROUP,SAMPLE
Taxon,,,,
"Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Ignisphaera",7,MID1,EXTRACTION CONTROL,NaN
"Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Pyrodictiaceae Pyrolobus",2,MID1,EXTRACTION CONTROL,NaN
"Archaea ""Crenarchaeota"" Thermoprotei Sulfolobales Sulfolobaceae Stygiolobus",3,MID1,EXTRACTION CONTROL,NaN
"Archaea ""Crenarchaeota"" Thermoprotei Thermoproteales Thermofilaceae Thermofilum",3,MID1,EXTRACTION CONTROL,NaN
"Archaea ""Euryarchaeota"" ""Methanomicrobia"" Methanocellales Methanocellaceae Methanocella",7,MID1,EXTRACTION CONTROL,NaN
"Archaea ""Euryarchaeota"" ""Methanomicrobia"" Methanosarcinales Methanosarcinaceae Methanimicrococcus",1,MID1,EXTRACTION CONTROL,NaN
"Archaea ""Euryarchaeota"" ""Methanomicrobia"" Methanosarcinales Methermicoccaceae Methermicoccus",1,MID1,EXTRACTION CONTROL,NaN
"Archaea ""Euryarchaeota"" Archaeoglobi Archaeoglobales Archaeoglobaceae Ferroglobus",1,MID1,EXTRACTION CONTROL,NaN
"Archaea ""Euryarchaeota"" Archaeoglobi Archaeoglobales Archaeoglobaceae Geoglobus",1,MID1,EXTRACTION CONTROL,NaN


### Checking for categorical variables independence using the Chi-Square test

To illustrate how to use the chi-square test to determine correlation between categorical attributes, let's use another dataset that has more potential for correlated attributes: the Census (Adult Income) Dataset. 

### The Adult Income Dataset

The <a href="https://www.kaggle.com/wenruliu/adult-income-dataset">Adult Income Dataset</a> includes data about an individual’s annual income. Intuitively, income is influenced by the individual’s education level, age, gender, occupation, and etc. The dataset contains 14 columns detailing attributes related to the demographics and other features that describe a person. The target attribute, Income, is divide into two classes: <=50K and >50K. A description of the attributes follows:

<b>age</b>: continuous.
<br><b>workclass</b>: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
<br><b>fnlwgt</b>: continuous.
<br><b>education</b>: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
<br><b>education-num</b>: continuous.
<br><b>marital-status</b>: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
<br><b>occupation</b>: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
<br><b>relationship</b>: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
<br><b>race</b>: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
<br><b>gender</b>: Female, Male.
<br><b>capital-gain</b>: continuous.
<br><b>capital-loss</b>: continuous.
<br><b>hours-per-week</b>: continuous.
<br><b>native-country</b>: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
<br><b>income</b>: >50K, <=50K

Let's import some important modules and then import the data.

In [37]:
import numpy as np
import pandas as pd
import sklearn.preprocessing as preprocessing
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder

income_df = pd.read_csv("data/income_data.csv")

# List attribute names
print (list(income_df))

# Display the shape of the dataset (#rows, #columns)
print (income_df.shape)

['age', 'workclass', 'fnlwgt', 'education', 'educational-num', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']
(48842, 15)


You may want to inspect some records from the dataset to get a feel of the attributes and their values.

In [38]:
#income_df.head() # first 5 rows
income_df.sample(10) # random 10 rows

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
20498,20,?,307149,Some-college,10,Never-married,?,Own-child,White,Female,0,0,35,United-States,<=50K
15504,44,Private,438696,Bachelors,13,Never-married,Sales,Not-in-family,White,Male,0,0,45,United-States,<=50K
41397,29,Private,275110,Some-college,10,Separated,Handlers-cleaners,Not-in-family,Black,Male,0,0,42,United-States,<=50K
15644,29,Private,109989,HS-grad,9,Never-married,Sales,Not-in-family,White,Male,0,0,40,United-States,<=50K
2936,21,State-gov,82847,Some-college,10,Never-married,Exec-managerial,Not-in-family,White,Female,0,0,38,United-States,<=50K
43936,41,Private,106679,Masters,14,Never-married,Exec-managerial,Not-in-family,White,Male,27828,0,50,United-States,>50K
19437,33,Private,94235,Prof-school,15,Never-married,Prof-specialty,Own-child,White,Male,0,0,42,United-States,>50K
12524,48,Private,377401,Some-college,10,Divorced,Sales,Not-in-family,White,Male,0,0,40,United-States,>50K
7861,48,Private,233511,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,>50K
36908,34,Private,125279,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States,<=50K


Let's explore if there is a correlation with gender and workclass. First, we need to build the contingency matrix for the two attributes:

In [39]:
contengency_table = pd.crosstab(income_df["workclass"],income_df["gender"], margins= True)
contengency_table

gender,Female,Male,All
workclass,,,
?,1270,1529,2799
Federal-gov,452,980,1432
Local-gov,1258,1878,3136
Never-worked,3,7,10
Private,11599,22307,33906
Self-emp-inc,211,1484,1695
Self-emp-not-inc,629,3233,3862
State-gov,763,1218,1981
Without-pay,7,14,21


Let's state the hypotheses:

<b>Null hypothesis:</b> There is no statistically significant relationship between gender and workclass.

<b>Alternative hypothesis:</b> There is a statistically significant relationship between the gender and workclass.

Each cell in the table represents the frequency count for the intersection of both values. The intersection of "male" and "federal-gov" represents the number of men who work in jobs related to the federal government.

Now let's calculate the chi-square test using SciPy. The `chi2_contengency()` method is applied to a two dimensional array representing the actual attribute values, and it automatically computes the contingency matrix and outputs three numbers: the chi2 value (difference between observed and expected counts), the p-value, and the degrees of freedom. What you want to check is the second value outputted, which is the p-value. You want it to be smaller than 0.05 so that the null hypothesis can be rejected.   

In [40]:
# Take the previously produced contingency matrix and apply the chi2 test method to it
#st.chi2_contingency(_)
st.chi2_contingency(contengency_table)

(1125.2572181055543,
 1.137527721203814e-227,
 18,
 array([[9.27918758e+02, 1.87108124e+03, 2.79900000e+03],
        [4.74733713e+02, 9.57266287e+02, 1.43200000e+03],
        [1.03964031e+03, 2.09635969e+03, 3.13600000e+03],
        [3.31517956e+00, 6.68482044e+00, 1.00000000e+01],
        [1.12404478e+04, 2.26655522e+04, 3.39060000e+04],
        [5.61922935e+02, 1.13307706e+03, 1.69500000e+03],
        [1.28032235e+03, 2.58167765e+03, 3.86200000e+03],
        [6.56737071e+02, 1.32426293e+03, 1.98100000e+03],
        [6.96187707e+00, 1.40381229e+01, 2.10000000e+01],
        [1.61920000e+04, 3.26500000e+04, 4.88420000e+04]]))

<span style="color:red"><b>Do we reject the null hypothesis based on the previous results?</b></span>

<span style="color:red"><b>In the previous output, and based on your understanding of the chi2 test from the lecture, are the degrees of freedom produced correct? If no, explain why and fix the problem and recompute the chi2 test.</b></span>


No, we don't because the p-value > 0.05.
Yes, the degrees of freedom are correct because dof = (r-1)(c-1).  So (10-1)(3-1)=18

## <span style="color:red">Assignment Mini Challenge 2</span>

<span style="color:red"><b>Write code that will compute the correlation between the `race` and `education` attributes. Is there a correlation or are the attributes independent?</b></span>

In [41]:
income_df['race'] = income_df['race'].astype('category').cat.codes
income_df['education'] = income_df['education'].astype('category').cat.codes
corr_data = income_df["race"].corr(income_df["education"])
corr_data

0.01324978686944515

## Part 2 - Engineering Features






Feature engineering is not a deterministic task, but some prelimenary things can still be done in a straigtforward way. In the following cells we will perform encoding, discretization, and aggregation tasks as part of feature engineering.

We are still using the Adult Income Dataset. Let's investigate descriptions of individual attributes.

In [42]:
def summerize_data(df):
    for column in df.columns:
        print (column)
        if df.dtypes[column] == np.object: # Categorical data
            print (df[column].value_counts())
        else:
            print (df[column].describe()) 
            
        print ('\n')
    
summerize_data(income_df)

age
count    48842.000000
mean        38.643585
std         13.710510
min         17.000000
25%         28.000000
50%         37.000000
75%         48.000000
max         90.000000
Name: age, dtype: float64


workclass
Private             33906
Self-emp-not-inc     3862
Local-gov            3136
?                    2799
State-gov            1981
Self-emp-inc         1695
Federal-gov          1432
Without-pay            21
Never-worked           10
Name: workclass, dtype: int64


fnlwgt
count    4.884200e+04
mean     1.896641e+05
std      1.056040e+05
min      1.228500e+04
25%      1.175505e+05
50%      1.781445e+05
75%      2.376420e+05
max      1.490400e+06
Name: fnlwgt, dtype: float64


education
count    48842.000000
mean        10.288420
std          3.874492
min          0.000000
25%          9.000000
50%         11.000000
75%         12.000000
max         15.000000
Name: education, dtype: float64


educational-num
count    48842.000000
mean        10.078089
std          2.570973


You can use the `OneHotCategoricalEncoder()` method to perform one-hot encoding, but you need to install the feature-engine package using:

    pip install feature-engine

at the command prompt. Let's try this method.

In [43]:
!pip install feature-engine

In [44]:
# Try one hot encoding

# Copy the original data
encoded_income_df = income_df.copy()

# Select the numeric columns
numeric_subset = income_df.select_dtypes('number')
# Select the categorical columns
categorical_subset = income_df.select_dtypes('object')
# Import feature-engine
from feature_engine.categorical_encoders import OneHotCategoricalEncoder

# Define encoder method
encoder = OneHotCategoricalEncoder()

# Apply encoder to categorical subset, except income
onehot_categorical_subset = encoder.fit_transform(categorical_subset[categorical_subset.columns.drop("income")])

# Beauty of feature-engine encoder method is it already returns a dataframe, so we need not worry about conversions
# Concatenate and reconstruct new dataset
onehot_encoded_data = pd.concat([numeric_subset, onehot_categorical_subset, income_df["income"]], axis = 1)

# display sample 10 records
onehot_encoded_data.sample(10)

,age,fnlwgt,education,educational-num,race,capital-gain,capital-loss,hours-per-week,workclass_Private,workclass_Local-gov,...,native-country_Ecuador,native-country_Yugoslavia,native-country_Hungary,native-country_Hong,native-country_Greece,native-country_Trinadad&Tobago,native-country_Outlying-US(Guam-USVI-etc),native-country_France,native-country_Holand-Netherlands,income
33002,28,147560,15,10,4,0,1902,55,1,0,...,0,0,0,0,0,0,0,0,0,>50K
27771,27,218184,15,10,2,0,0,40,1,0,...,0,0,0,0,0,0,0,0,0,<=50K
23691,21,226145,15,10,4,0,0,30,1,0,...,0,0,0,0,0,0,0,0,0,<=50K
46316,27,401508,11,9,2,0,0,40,1,0,...,0,0,0,0,0,0,0,0,0,<=50K
6644,30,61989,15,10,4,0,0,5,0,0,...,0,0,0,0,0,0,0,0,0,<=50K
18334,24,233777,15,10,4,0,0,50,1,0,...,0,0,0,0,0,0,0,0,0,<=50K
27457,31,349148,11,9,4,0,0,40,1,0,...,0,0,0,0,0,0,0,0,0,>50K
2445,35,211440,7,12,4,0,0,45,1,0,...,0,0,0,0,0,0,0,0,0,<=50K
26599,35,287658,15,10,2,0,0,40,0,0,...,0,0,0,0,0,0,0,0,0,>50K
11687,31,220669,9,13,4,6849,0,40,0,1,...,0,0,0,0,0,0,0,0,0,<=50K


#### Discretization 

Now let's discretize some of the numerical attributes. We will work with the <b>age</b> and the <b>hours-per-week</b> as examples.

In [45]:
# Group the "age" column
age_group = [] # define array structure
for age in encoded_income_df["age"]:
    if age < 25:
        age_group.append("<25")
    elif 25 <= age <= 34:
        age_group.append("25-34")
    elif 34 < age <= 44:
        age_group.append("35-44")
    elif 44 < age <= 54:
        age_group.append("45-54")
    elif 54 < age <= 65:
        age_group.append("55-64")
    else:
        age_group.append("65 and over")
        
# Copy dataframe to keep original 
new_income_df = encoded_income_df.copy()
new_income_df["age_group"] = age_group
del new_income_df["age"]

# Same thing for "hours-per-week"
work_hours_per_week = []
for hours in encoded_income_df["hours-per-week"]:
    if hours < 16:
        work_hours_per_week.append("<16")
    elif 16 <= hours <= 32:
        work_hours_per_week.append("16-32")
    elif 32 < hours <= 48:
        work_hours_per_week.append("32-48")
    elif 48 < hours <= 60:
        work_hours_per_week.append("48-60")
    else:
        work_hours_per_week.append("60 and over")
        
new_income_df["work_hours_per_week"] = work_hours_per_week
del new_income_df["hours-per-week"]

new_income_df.head(10)

,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,native-country,income,age_group,work_hours_per_week
0,Private,226802,1,7,Never-married,Machine-op-inspct,Own-child,2,Male,0,0,United-States,<=50K,25-34,32-48
1,Private,89814,11,9,Married-civ-spouse,Farming-fishing,Husband,4,Male,0,0,United-States,<=50K,35-44,48-60
2,Local-gov,336951,7,12,Married-civ-spouse,Protective-serv,Husband,4,Male,0,0,United-States,>50K,25-34,32-48
3,Private,160323,15,10,Married-civ-spouse,Machine-op-inspct,Husband,2,Male,7688,0,United-States,>50K,35-44,32-48
4,?,103497,15,10,Never-married,?,Own-child,4,Female,0,0,United-States,<=50K,<25,16-32
5,Private,198693,0,6,Never-married,Other-service,Not-in-family,4,Male,0,0,United-States,<=50K,25-34,16-32
6,?,227026,11,9,Never-married,?,Unmarried,2,Male,0,0,United-States,<=50K,25-34,32-48
7,Self-emp-not-inc,104626,14,15,Married-civ-spouse,Prof-specialty,Husband,4,Male,3103,0,United-States,>50K,55-64,16-32
8,Private,369667,15,10,Never-married,Other-service,Unmarried,4,Female,0,0,United-States,<=50K,<25,32-48
9,Private,104996,5,4,Married-civ-spouse,Craft-repair,Husband,4,Male,0,0,United-States,<=50K,55-64,<16


There is an easier way in python than using if else statements. Pandas `cut` function can be used to group continuous or countable data in to bins.

In [46]:
# Take another copy of the original dataset
new_income_df2 = encoded_income_df.copy()

# Cut the age attribute into intervals
age_group2 = pd.cut(new_income_df2.age, [20,40,60,80])

# You can label the intervals for more meaningful representation
#age_group2 = pd.cut(new_income_df2.age, [20,40,60,80],labels=['young','middle-aged','old','senior'])

new_income_df2["age_group"] = age_group2
del new_income_df2["age"]

new_income_df2.head(10)

,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income,age_group
0,Private,226802,1,7,Never-married,Machine-op-inspct,Own-child,2,Male,0,0,40,United-States,<=50K,"(20.0, 40.0]"
1,Private,89814,11,9,Married-civ-spouse,Farming-fishing,Husband,4,Male,0,0,50,United-States,<=50K,"(20.0, 40.0]"
2,Local-gov,336951,7,12,Married-civ-spouse,Protective-serv,Husband,4,Male,0,0,40,United-States,>50K,"(20.0, 40.0]"
3,Private,160323,15,10,Married-civ-spouse,Machine-op-inspct,Husband,2,Male,7688,0,40,United-States,>50K,"(40.0, 60.0]"
4,?,103497,15,10,Never-married,?,Own-child,4,Female,0,0,30,United-States,<=50K,NaN
5,Private,198693,0,6,Never-married,Other-service,Not-in-family,4,Male,0,0,30,United-States,<=50K,"(20.0, 40.0]"
6,?,227026,11,9,Never-married,?,Unmarried,2,Male,0,0,40,United-States,<=50K,"(20.0, 40.0]"
7,Self-emp-not-inc,104626,14,15,Married-civ-spouse,Prof-specialty,Husband,4,Male,3103,0,32,United-States,>50K,"(60.0, 80.0]"
8,Private,369667,15,10,Never-married,Other-service,Unmarried,4,Female,0,0,40,United-States,<=50K,"(20.0, 40.0]"
9,Private,104996,5,4,Married-civ-spouse,Craft-repair,Husband,4,Male,0,0,10,United-States,<=50K,"(40.0, 60.0]"


## <span style="color:red">Assignment Mini Challenge 3</span>

It is possible to use `cut` with quantiles instead of intervals (depth-based binning, # of ovservations in bin instead of interval of values). Read relevant documentation of the method on the Web.

In the Adult Income dataset, there is an `age` attribute, which is numerical.

<span style="color:red"><b>Write code that will do the following:</b></span>
    
<span style="color:red"><b>1- Apply quantile cutting to the `age` attribute and construct new dataset.</b></span>

<span style="color:red"><b>2- Apply one-hot encoding again to the discretized attribute.</b></span>

In [47]:
# Take another copy of the original dataset
new_income_df3 = encoded_income_df.copy()
# Use the cut method
new_income_df3['new_age'] = pd.qcut(new_income_df3['age'], q=4)

new_income_df3.head(10)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income,new_age
0,25,Private,226802,1,7,Never-married,Machine-op-inspct,Own-child,2,Male,0,0,40,United-States,<=50K,"(16.999, 28.0]"
1,38,Private,89814,11,9,Married-civ-spouse,Farming-fishing,Husband,4,Male,0,0,50,United-States,<=50K,"(37.0, 48.0]"
2,28,Local-gov,336951,7,12,Married-civ-spouse,Protective-serv,Husband,4,Male,0,0,40,United-States,>50K,"(16.999, 28.0]"
3,44,Private,160323,15,10,Married-civ-spouse,Machine-op-inspct,Husband,2,Male,7688,0,40,United-States,>50K,"(37.0, 48.0]"
4,18,?,103497,15,10,Never-married,?,Own-child,4,Female,0,0,30,United-States,<=50K,"(16.999, 28.0]"
5,34,Private,198693,0,6,Never-married,Other-service,Not-in-family,4,Male,0,0,30,United-States,<=50K,"(28.0, 37.0]"
6,29,?,227026,11,9,Never-married,?,Unmarried,2,Male,0,0,40,United-States,<=50K,"(28.0, 37.0]"
7,63,Self-emp-not-inc,104626,14,15,Married-civ-spouse,Prof-specialty,Husband,4,Male,3103,0,32,United-States,>50K,"(48.0, 90.0]"
8,24,Private,369667,15,10,Never-married,Other-service,Unmarried,4,Female,0,0,40,United-States,<=50K,"(16.999, 28.0]"
9,55,Private,104996,5,4,Married-civ-spouse,Craft-repair,Husband,4,Male,0,0,10,United-States,<=50K,"(48.0, 90.0]"


In [48]:
#one hot encoded

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(new_income_df3['new_age'])
print(integer_encoded)

# binary encode
onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)

[0 2 0 ... 3 0 3]
[[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]]


## <span style="color:red">Assignment Mini Challenge 4</span>

In the Adult Income dataset, there is an `hours_per_week` attribute, which is numerical.

<span style="color:red"><b>Write code that will do the following:</b></span>

<span style="color:red"><b>1- Transform this attribute into a categorical attribute by dividing the range of values into bins with the following lables: "0-9", "10-19", "20-29", "30-39", "40-49", "50+". Rename the new attribute as `working_hours_categories`. </b></span>

<span style="color:red"><b>2- Perform the chi2 test to find if the discretized attribute `working_hours_categories` and `gender` are correlated.</b></span>


In [49]:
# Take another copy of the original dataset
new_income_df4 = encoded_income_df.copy()

# Use the cut method
new_income_df4['working_hours_categories'] = pd.cut(new_income_df4['hours-per-week']
                                                     , [0, 9, 19, 29, 39, 49, 100], labels = ["0-9", "10-19", "20-29", "30-39", "40-49", "50+"])

new_income_df4.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income,working_hours_categories
0,25,Private,226802,1,7,Never-married,Machine-op-inspct,Own-child,2,Male,0,0,40,United-States,<=50K,40-49
1,38,Private,89814,11,9,Married-civ-spouse,Farming-fishing,Husband,4,Male,0,0,50,United-States,<=50K,50+
2,28,Local-gov,336951,7,12,Married-civ-spouse,Protective-serv,Husband,4,Male,0,0,40,United-States,>50K,40-49
3,44,Private,160323,15,10,Married-civ-spouse,Machine-op-inspct,Husband,2,Male,7688,0,40,United-States,>50K,40-49
4,18,?,103497,15,10,Never-married,?,Own-child,4,Female,0,0,30,United-States,<=50K,30-39


In [50]:
#2 perferom the chi2 test
contengency_table = pd.crosstab(new_income_df4["working_hours_categories"],new_income_df4["gender"], margins= True)

st.chi2_contingency(contengency_table)

(62764.241398436716,
 0.0,
 12,
 array([[ 1720.74394988,  3469.75605012,  5190.5       ],
        [ 1231.09192908,  2482.40807092,  3713.5       ],
        [ 5144.16412104, 10372.83587896, 15517.        ],
        [ 1507.74366324,  3040.25633676,  4548.        ],
        [ 4670.09344417,  9416.90655583, 14087.        ],
        [ 1918.16289259,  3867.83710741,  5786.        ],
        [16192.        , 32650.        , 48842.        ]]))

#### Aggregation

Now let's try to aggregate numerical values according to a categorical attribute. If there was a time attribute then aggregation could have been performed on different time intervals. For the dataset we have it is sufficient to apply aggregation over the workclass attribute for now. We will use the `groupby` function. Then, it is possible to compute aggregate values (e.g. mean) per workclass group for the numerical attributes.

In [51]:
# We will work with the original dataset income_df
# Group workclass attribute by its categorical values
grouped_income = income_df.groupby(["workclass"])

# Compute mean per group using agg function
grouped_income.agg(np.mean).head()

,age,fnlwgt,education,educational-num,race,capital-gain,capital-loss,hours-per-week
workclass,,,,,,,,
?,40.141122,187254.773491,9.961058,9.251161,3.603073,502.946052,67.634870,31.812433
Federal-gov,42.577514,183590.028631,11.052374,10.937151,3.409916,923.287709,108.884078,41.513268
Local-gov,41.676020,190161.134885,10.603635,11.032207,3.576849,798.228635,102.124043,40.847258
Never-worked,19.900000,215033.300000,6.000000,7.500000,3.400000,0.000000,0.000000,28.900000
Private,36.903144,192669.212499,10.174512,9.875715,3.674069,896.135374,80.768478,40.273137


The `agg` function intelligently ignores categorical attributes.

## <span style="color:red">Assignment Mini Challenge 5</span>

<span style="color:red"><b>Why did we not apply the aggregation function on the encoded dataset?</b></span>

``GroupBy`` has its own associated ``aggregate()`` method which allows for more flexibility and aggregate different columns using different aggregation functions in one shot. Look up how this method works, 

<span style="color:red"><b>Write code that uses the ``aggregate()`` to aggregate the `age` column by mean, the `fnlwgt` column by min, the `education-num` by max, the `capital-gain` by max, the `capital-loss` by max, and the `hours-per-week` by mean. </b></span>

In [52]:
age = income_df.groupby(['age'])
age.agg(np.mean).head()

,fnlwgt,education,educational-num,race,capital-gain,capital-loss,hours-per-week
age,,,,,,,
17,179157.852101,1.484034,6.685714,3.784874,81.245378,33.302521,21.137815
18,193421.075406,7.118329,8.220418,3.718097,76.163573,37.852668,25.745940
19,204916.851852,11.471985,9.086420,3.723647,126.797721,42.579297,30.560304
20,198076.513028,12.231806,9.341420,3.722372,66.937107,35.743935,32.432165
21,200227.520985,12.135949,9.430657,3.692518,169.385949,40.899635,34.252737


In [53]:
fnlwgt = income_df.groupby(['fnlwgt'])
fnlwgt.agg(np.min).head()

,age,workclass,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
fnlwgt,,,,,,,,,,,,,,
12285,25,?,15,10,Never-married,?,Not-in-family,0,Female,0,0,20,United-States,<=50K
13492,40,State-gov,11,9,Married-civ-spouse,Protective-serv,Husband,0,Male,0,0,84,United-States,<=50K
13769,23,Private,8,10,Divorced,Exec-managerial,Husband,0,Male,0,0,30,United-States,<=50K
13862,32,?,11,9,Never-married,?,Not-in-family,0,Female,0,0,38,United-States,<=50K
14878,46,Local-gov,14,15,Married-civ-spouse,Prof-specialty,Husband,1,Male,0,0,40,Philippines,>50K


In [54]:
educationNum = income_df.groupby(['educational-num'])
educationNum.agg(np.max).head()

,age,workclass,fnlwgt,education,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
educational-num,,,,,,,,,,,,,,
1,77,State-gov,572751,13,Widowed,Transport-moving,Wife,4,Male,41310,1719,75,United-States,>50K
2,90,State-gov,795830,3,Widowed,Transport-moving,Wife,4,Male,7688,2603,96,Vietnam,>50K
3,84,State-gov,684015,4,Widowed,Transport-moving,Wife,4,Male,99999,2603,99,Yugoslavia,>50K
4,90,Without-pay,750972,5,Widowed,Transport-moving,Wife,4,Male,10566,3900,99,Yugoslavia,>50K
5,90,State-gov,814850,6,Widowed,Transport-moving,Wife,4,Male,99999,2231,99,Yugoslavia,>50K


In [55]:
capitalGain = income_df.groupby(['capital-gain'])
capitalGain.agg(np.max).head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-loss,hours-per-week,native-country,income
capital-gain,,,,,,,,,,,,,,
0,90,Without-pay,1490400,15,16,Widowed,Transport-moving,Wife,4,Male,4356,99,Yugoslavia,>50K
114,39,State-gov,354591,15,11,Widowed,Transport-moving,Unmarried,4,Male,0,55,United-States,<=50K
401,90,Self-emp-not-inc,258973,15,13,Widowed,Sales,Not-in-family,4,Male,0,20,United-States,<=50K
594,57,State-gov,675421,15,16,Widowed,Sales,Unmarried,4,Male,0,60,United-States,<=50K
914,55,State-gov,210424,15,14,Widowed,Prof-specialty,Unmarried,4,Male,0,50,United-States,<=50K


In [56]:
capitalLoss = income_df.groupby(['capital-loss'])
capitalLoss.agg(np.max).head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,hours-per-week,native-country,income
capital-loss,,,,,,,,,,,,,,
0,90,Without-pay,1490400,15,16,Widowed,Transport-moving,Wife,4,Male,99999,99,Yugoslavia,>50K
155,28,Private,140845,0,6,Never-married,Machine-op-inspct,Not-in-family,4,Female,0,40,United-States,<=50K
213,42,Private,262409,15,14,Divorced,Tech-support,Unmarried,4,Female,0,45,United-States,<=50K
323,52,Private,198774,12,14,Widowed,Prof-specialty,Unmarried,4,Male,0,45,United-States,<=50K
419,68,Private,501172,15,10,Married-civ-spouse,Transport-moving,Husband,4,Male,0,20,United-States,<=50K


In [57]:
hoursPerWeek = income_df.groupby(['hours-per-week'])
hoursPerWeek.agg(np.mean).head()

,age,fnlwgt,education,educational-num,race,capital-gain,capital-loss
hours-per-week,,,,,,,
1,57.111111,183758.777778,9.148148,10.000000,3.296296,120.629630,142.444444
2,54.490566,194164.396226,9.924528,10.377358,3.698113,858.849057,105.735849
3,52.728814,184890.525424,9.440678,9.847458,3.898305,738.711864,103.457627
4,46.821429,165712.214286,9.785714,9.595238,3.619048,489.678571,19.619048
5,47.778947,173862.400000,8.715789,9.726316,3.757895,297.484211,55.178947


## <span style="color:red">Assignment Bonus Challenge</span>

<span style="color:red"><b>For the Adult Income Dataset:</b></span>

<span style="color:red"><b>1- Try to come up with at least one indicator feature based on threshold, multiple attributes, or multiple categorical values in an attribute</b></span>

<span style="color:red"><b>2- Try to come up with at least one meaningful interaction feature.</b></span>